## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd
import glob
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import requests
import time

## Data Read-in

In [2]:
csv_files = glob.glob('*.csv')

df_list = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df_list.append(df)
    
df = pd.concat(df_list,ignore_index=True)

print(len(df))

3029


In [3]:
gdf = gpd.read_file('Miami-Dade_Boundary.geojson')

## Previous month's values

Get values from here:

https://docs.google.com/spreadsheets/d/1GWlqTIzCB29pDVDjd-iBVejVGGsLiAd0P6fq4LM83WI/edit?usp=sharing

In [4]:
print('Input total condo units sold previous (not last) month:')
total_condos_sold_2nd_month = input()

Input total condo units sold previous (not last) month:
810


In [5]:
print('Input total sales volume ($$$ million) from previous (not last) month:')
sales_volume_2nd_month = input()

Input total sales volume ($$$ million) from previous (not last) month:
624000000


In [6]:
print('Input median sales price from previous (not last) month:')
median_sales_price_2nd_month = input()

Input median sales price from previous (not last) month:
407500


In [7]:
print('Input median psf from previous (not last) month:')
median_psf_2nd_month = input()

Input median psf from previous (not last) month:
416


## Data Clean

In [8]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [9]:
df = df.dropna(subset=['SOLD DATE'])

In [10]:
# Get the current date
current_date = datetime.now()

# Calculate the first day of the current month
first_day_of_current_month = current_date.replace(day=1)

# Get the last day of the previous month by subtracting one day from the first day of the current month
last_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# Get the month name of the previous month
previous_month_name = last_day_of_previous_month.strftime("%B")

# ---- Get the name of the month, as a string, from two months ago.
# ---- Meaning, if the current month is January, return November.

# Get the first day of the previous month by subtracting one day from the first day of the current month
first_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# Get the first day of the month before the last by subtracting one day from the first day of the previous month
first_day_of_month_before_last = first_day_of_previous_month.replace(day=1) - timedelta(days=1)

# Get the month name of the month before the last
month_before_last_name = first_day_of_month_before_last.strftime("%B")

print(f'Report month: {previous_month_name} ---- Month before report month: {month_before_last_name}')

Report month: August ---- Month before report month: July


In [11]:
# Define list of desired months (excluding current month)
desired_months = [previous_month_name]

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [12]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    761
Name: count, dtype: int64
-------
$/SQUARE FEET
False    757
True       4
Name: count, dtype: int64
-------
YEAR BUILT
False    761
Name: count, dtype: int64
-------


In [13]:
sorted_df = df_filtered.sort_values(by='YEAR BUILT', ascending=False)
second_newest_building = sorted_df.iloc[2]
print(second_newest_building['URL'])

https://www.redfin.com/FL/Miami/227-NE-2nd-St-33132/unit-1508/home/184157804


In [14]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [15]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [16]:
df_filtered = df_filtered.drop_duplicates().reset_index(drop=True)

In [17]:
# df_filtered = df_filtered[df_filtered['ADDRESS'] != '1901 Brickell Ave Unit B109']

In [17]:
df_filtered.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [18]:
# df_filtered = df_filtered.drop(index=711)

In [27]:
df_filtered.sort_values(by='PRICE',ascending=True)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
182,PAST SALE,August-1-2024,Condo/Co-op,18801 NE 2nd Ave #1021,Miami,FL,33179.0,92500.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/18801-NE-2nd-A...,MARMLS,A11514727,N,Y,25.947718,-80.196856
173,PAST SALE,August-20-2024,Condo/Co-op,160 NE 203rd Ter #12,Miami Gardens,FL,33179.0,94000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/160-NE-203rd-T...,MARMLS,A11527763,N,Y,25.961793,-80.199898
152,PAST SALE,August-26-2024,Condo/Co-op,160 NE 203rd Ter #22,Miami Gardens,FL,33179.0,115000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/160-NE-203rd-T...,MARMLS,A11601210,N,Y,25.961793,-80.199898
171,PAST SALE,August-20-2024,Condo/Co-op,5300 NW 87th Ave #1107,Doral,FL,33178.0,116000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Doral/5300-NW-87th-A...,MARMLS,A11547776,N,Y,25.820709,-80.338168
136,PAST SALE,August-8-2024,Condo/Co-op,2861 Leonard Dr Unit F509,Aventura,FL,33160.0,125000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2861-Leonar...,MARMLS,A11590422,N,Y,25.941858,-80.144472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,PAST SALE,August-22-2024,Condo/Co-op,2216 Fisher Island Dr #3106,Miami Beach,FL,33109.0,6119340.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2216-Fis...,MARMLS,A11598657,N,Y,25.758920,-80.143128
105,PAST SALE,August-8-2024,Condo/Co-op,18501 Collins Ave #2201,Sunny Isles Beach,FL,33160.0,7200000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11601770,N,Y,25.947720,-80.120228
113,PAST SALE,August-20-2024,Condo/Co-op,1000 Biscayne Blvd #4202,Miami,FL,33132.0,7300000.0,4.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Biscayne-...,MARMLS,A11589086,N,Y,25.784171,-80.190078
102,PAST SALE,August-27-2024,Condo/Co-op,1000 Biscayne Blvd #1202,Miami,FL,33132.0,7900000.0,5.0,6.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Biscayne-...,MARMLS,A11635953,N,Y,25.784171,-80.190078


In [21]:
print(df_filtered['URL'].iloc[182])

https://www.redfin.com/FL/Miami/18801-NE-2nd-Ave-33179/unit-1021/home/43012166


In [22]:
# Correct the prices, if needed
df_filtered.at[140,'PRICE']=(241_000)


# Correct the psf, if needed
df_filtered.at[140,'$/SQUARE FEET']=(241_000/1_000)

In [26]:
df_filtered = df_filtered.drop(axis=0,index=175)

In [21]:
### If needed, drop the row with the lowest price
# min_value_index = df_filtered['PRICE'].idxmin()
# df_filtered = df_filtered.drop(min_value_index)

In [28]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
167,185000.0,1800 NE 114th St #1811,Miami,95.0
166,151000.0,1670 NE 191st St Unit 212-3,Miami,115.0
708,230000.0,1800 NE 114th St #1206,Miami,118.0
182,92500.0,18801 NE 2nd Ave #1021,Miami,131.0
173,94000.0,160 NE 203rd Ter #12,Miami Gardens,136.0
168,169500.0,492 NW 165th St Rd Unit C609,Miami,144.0
152,115000.0,160 NE 203rd Ter #22,Miami Gardens,148.0
172,154000.0,1680 NE 191st St Unit 108-2,Miami,149.0
139,200000.0,1401 NE Miami Gardens Dr #490,Miami,152.0
164,129000.0,20230 NE 2nd Ave Unit U8,Miami Gardens,154.0


In [23]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [29]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [30]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [31]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [32]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [33]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [34]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"{previous_month_name} 2024 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

folium.GeoJson(gdf,tooltip='Miami-Dade County',name='Miami-Dade County').add_to(m)

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
# m

In [35]:
m.save('index.html')

## Summary Info

In [36]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [37]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [38]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [39]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami-Beach/1027-Pennsylvania-Ave-33139/unit-201/home/43271790


## Collect Agent Information

In [43]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [62]:
max_price_index = df_filtered['PRICE'].idxmax()
min_price_index = df_filtered['PRICE'].idxmin()

max_price_psf_index = df_filtered['$/SQUARE FEET'].idxmax()
min_price_psf_index = df_filtered['$/SQUARE FEET'].idxmin()

max_year_built_index = df_filtered['YEAR BUILT'].idxmax()
min_year_built_index = df_filtered['YEAR BUILT'].idxmin()

In [63]:
index_list = [max_price_index,min_price_index,
             max_price_psf_index,min_price_psf_index,
             max_year_built_index,min_year_built_index]

In [56]:
df_filtered.reset_index(inplace=True,drop=True)

In [65]:
# for index, row in df_filtered.iterrows():
#     if index in index_list:
#         print(f'{index} found!')
#         print(f'{df_filtered.iloc[index]}')

In [66]:
response_list = []

for index, row in df_filtered.iterrows():
    if index in index_list:
        response_list.append(process_response(row['URL']))

In [67]:
response_df = pd.DataFrame(response_list)

In [68]:
merged_df = pd.merge(left=df_filtered,left_on='URL',right=response_df, right_on='url')

In [127]:
test_df = pd.merge(left=df_filtered,left_on='URL',right=response_df, right_on='url',how='outer')

In [134]:
test_df.list_broker_name_2.unique()

array([nan, "One Sotheby's International Re",
       'Keller Williams Realty Miami Portfolio Collection',
       'EXP Realty LLC', None, 'Compass Florida, LLC'], dtype=object)

In [80]:
merged_df.columns

Index(['index', 'SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL',
       'LATITUDE', 'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK', 'FULL_ADDRESS', 'url',
       'list_agent_name_1', 'list_broker_name_1', 'list_agent_name_2',
       'list_broker_name_2', 'buy_agent_name_1', 'buy_broker_name_1',
       'buy_agent_name_2', 'buy_broker_name_2'],
      dtype='object')

In [89]:
print(f"{ME}{BR}{merged_df.loc[merged_df['PRICE'].idxmax()]['LOCATION']}, {merged_df.loc[merged_df['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${merged_df.loc[merged_df['PRICE'].idxmax()]['PRICE']:,.0f} | ${merged_df.loc[merged_df['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {merged_df.loc[merged_df['PRICE'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {merged_df.loc[merged_df['PRICE'].idxmax()]['list_agent_name_1']} with {merged_df.loc[merged_df['PRICE'].idxmax()]['list_broker_name_1']} and {merged_df.loc[merged_df['PRICE'].idxmax()]['list_agent_name_2']} with {merged_df.loc[merged_df['PRICE'].idxmax()]['list_broker_name_2']} | Buyer's agents: {merged_df.loc[merged_df['PRICE'].idxmax()]['buy_agent_name_1']} with {merged_df.loc[merged_df['PRICE'].idxmax()]['buy_broker_name_1']} and {merged_df.loc[merged_df['PRICE'].idxmax()]['buy_agent_name_2']} with {merged_df.loc[merged_df['PRICE'].idxmax()]['buy_broker_name_2']}")
print(f"{LE}{BR}{merged_df.loc[merged_df['PRICE'].idxmin()]['LOCATION']}, {merged_df.loc[merged_df['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${merged_df.loc[merged_df['PRICE'].idxmin()]['PRICE']:,.0f} | ${merged_df.loc[merged_df['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {merged_df.loc[merged_df['PRICE'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {merged_df.loc[merged_df['PRICE'].idxmin()]['list_agent_name_1']} with {merged_df.loc[merged_df['PRICE'].idxmin()]['list_broker_name_1']} and {merged_df.loc[merged_df['PRICE'].idxmin()]['list_agent_name_2']} with {merged_df.loc[merged_df['PRICE'].idxmin()]['list_broker_name_2']} | Buyer's agents: {merged_df.loc[merged_df['PRICE'].idxmin()]['buy_agent_name_1']} with {merged_df.loc[merged_df['PRICE'].idxmin()]['buy_broker_name_1']} and {merged_df.loc[merged_df['PRICE'].idxmin()]['buy_agent_name_2']} with {merged_df.loc[merged_df['PRICE'].idxmin()]['buy_broker_name_2']}")

print(f"{MAX_PSF}{BR}{merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['LOCATION']}, {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['list_agent_name_1']} with {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['list_broker_name_1']} and {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['list_agent_name_2']} with {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['list_broker_name_2']} | Buyer's agents: {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['buy_agent_name_1']} with {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['buy_broker_name_1']} and {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['buy_agent_name_2']} with {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['LOCATION']}, {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['list_agent_name_1']} with {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['list_broker_name_1']} and {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['list_agent_name_2']} with {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['list_broker_name_2']} | Buyer's agents: {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['buy_agent_name_1']} with {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['buy_broker_name_1']} and {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['buy_agent_name_2']} with {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['buy_broker_name_2']}")

print(f"{Newest}{BR}{merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['LOCATION']}, {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['list_agent_name_1']} with {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['list_broker_name_1']} and {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['list_agent_name_2']} with {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['list_broker_name_2']} | Buyer's agents: {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['buy_agent_name_1']} with {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['buy_broker_name_1']} and {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['buy_agent_name_2']} with {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['buy_broker_name_2']}")
print(f"{Oldest}{BR}{merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['LOCATION']}, {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['list_agent_name_1']} with {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['list_broker_name_1']} and {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['list_agent_name_2']} with {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['list_broker_name_2']} | Buyer's agents: {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['buy_agent_name_1']} with {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['buy_broker_name_1']} and {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['buy_agent_name_2']} with {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['buy_broker_name_2']}")

Most Expensive
DINNER KEY, 2669 S Bayshore Dr Unit LPH-N Miami | Price $19,500,000 | $1,937 psf | Year built: 2016 | Listing agents: Chad Carroll with Compass Florida, LLC and Lisandra Martinez with Compass Florida, LLC | Buyer's agents: Malgozata Makovska Fuller with One Sotheby's International Re and None with None
Least Expensive
STAR LAKES ESTATES NO 10, 18801 NE 2nd Ave #1021 Miami | Price $92,500 | $131 psf | Year built: 1969 | Listing agents: Steve Menachem with United Realty Group Inc and None with None | Buyer's agents: Yessica Mejias with The Keyes Company and None with None
Highest Price Per Square Foot
CONTINUUM ON SOUTH BEACH, 100 S Pointe Dr #3205 Miami Beach | Price $5,650,000 | $3,021 psf | Year built: 2003 | Listing agents: Eddy Martinez with One Sotheby's International Re and Roland Ortiz with One Sotheby's International Re | Buyer's agents: Carla Ellek with Westlake Realty Group, LLC. and None with None
Lowest Price Per Square Foot
CRICKET CLUB CONDO, 1800 NE 114th S

## Map URL Snagger

In [90]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MD_condo_sales_month_ending_august_2024


## Get Summary Data

In [92]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 712
--------
Total sale price: $470,342,101
--------
Median sale price: $380,000
--------
Max sale price: $19,500,000
--------
Min sale price: $92,500
------------------------------------------------
PSF INFO
Max price per square foot: $3,021
--------
Min price per square foot: $95
--------
Median price per square foot: $388
------------------------------------------------
CONDO AGES
Newest building: 2024.0
----------
Oldest building: 1925.0
----------
Average building age: 1987.7893258426966


In [93]:
most_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']
most_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmax()]['CITY']

least_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']
least_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmin()]['CITY']

# GENERATE STORY TEMPLATE

# STOP! DID YOU UPDATE VALUES FROM THE PREVIOUS MONTH?

In [115]:
## Calculate the increase/descrease between both months
subject_month_sales_volume = df_filtered["PRICE"].sum()
prior_month_sales_volume = int(sales_volume_2nd_month)  # Example value, replace with actual value

subject_month_closings = len(df_filtered)
prior_month_closings = int(total_condos_sold_2nd_month)

# Subtract the smaller value from the larger one
if subject_month_sales_volume > prior_month_sales_volume:
    hed_rf = 'jumps'
    nut_graf = 'rising'
    social = 'higher'
    seo = 'increases'
elif prior_month_sales_volume > subject_month_sales_volume:
    hed_rf = 'drops'
    nut_graf = 'falling'
    social = 'lower'
    seo = 'decreases'
else:
    hed_rf = 'equals'
    nut_graf = 'equaling'
    social = 'equal'
    seo = "doesn't change"
      
if subject_month_closings > prior_month_closings:
    dek_rf = 'rises'
    graf = 'up'
elif prior_month_closings > subject_month_closings:
    dek_rf ='fell'
    graf = 'down'
else:
    dek_rf = 'equals'
    graf = 'flat'

In [95]:
subject_month_closings

712

In [96]:
prior_month_sales_volume

624000000

In [97]:
prior_month_closings

810

In [116]:
story_string = f'''
\033[1mHED:\033[0m Miami-Dade County condo sales dollar volume {hed_rf} in {previous_month_name} to ${round(df_filtered["PRICE"].sum()/1_000_000)}M 
\033[1mDEK:\033[0m Number of closings {dek_rf} to {len(df_filtered)} from {total_condos_sold_2nd_month} in {month_before_last_name}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County {previous_month_name} Condo Sales Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales volume {hed_rf} to ${round(df_filtered["PRICE"].sum()/1_000_000)} million in {previous_month_name}.
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m Miami-Dade County had ${round(df_filtered["PRICE"].sum()/1_000_000):,.0f} million in condo sales in {previous_month_name}, {social} than ${round(int(sales_volume_2nd_month)/1_000_000)} million from {month_before_last_name}, @afarence reports 
\033[1mART:\033[0m 

CHANGE ME

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, monthly condo sales, condos, condo market, {most_expensive_muni_condo_name.title()}, {most_expensive_muni_location}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m


Miami-Dade County’s NEWS PEG HERE.

{previous_month_name} condo sales totaled ${round(df_filtered["PRICE"].sum()/1_000_000)} million, {nut_graf} from ${int(sales_volume_2nd_month)/1_000_000:.0f} million in {month_before_last_name}. Brokers closed {len(df_filtered)} sales last month, {graf} from {total_condos_sold_2nd_month} sales in {month_before_last_name} and XXX in XXX, Multiple Listing Service data from Redfin show.

{previous_month_name} sale prices ranged from ${df_filtered["PRICE"].min():,.0f} to ${df_filtered["PRICE"].max()/1_000_000:,.0f} million, with a median sale price of ${df_filtered["PRICE"].median():,.0f}. The price per square foot ranged from ${df_filtered["$/SQUARE FEET"].min():,.0f} to ${df_filtered["$/SQUARE FEET"].max():,.0f}, with a median of ${df_filtered["$/SQUARE FEET"].median():,.0f} per square foot.

In {month_before_last_name}, sales closed with a median price of ${int(median_sales_price_2nd_month):,.0f}, and ${median_psf_2nd_month} per square foot.

A ${df_filtered["PRICE"].max()/1_000_000:,.0f} million closing at {most_expensive_muni_condo_name.title()} took the title of priciest sale last month. {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} sold for ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} per square foot after XX days on the market. XXX with XXX had the listing, and XXX with XXX represented the buyer.

The sale price was \033[1mmore/less\033[0m than {month_before_last_name}'s priciest sale. WHEN XXXX 

In contrast, {previous_month_name}'s cheapest sale was an {least_expensive_muni_condo_name} in {least_expensive_muni_location}, at {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']}. Unit XXX traded for ${df_filtered['PRICE'].min():,.0f} — or ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} per square foot — after XXX days on the market. XXX with XXX handled both sides of the deal.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="800" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of {previous_month_name}’s notable condo sales:
'''

In [44]:
# print(df_filtered['URL'].iloc[0])

In [125]:
print(story_string)

print(f"{ME}{BR}{merged_df.loc[merged_df['PRICE'].idxmax()]['LOCATION']}, {merged_df.loc[merged_df['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${merged_df.loc[merged_df['PRICE'].idxmax()]['PRICE']:,.0f} | ${merged_df.loc[merged_df['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {merged_df.loc[merged_df['PRICE'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {merged_df.loc[merged_df['PRICE'].idxmax()]['list_agent_name_1']} with {merged_df.loc[merged_df['PRICE'].idxmax()]['list_broker_name_1']} and {merged_df.loc[merged_df['PRICE'].idxmax()]['list_agent_name_2']} with {merged_df.loc[merged_df['PRICE'].idxmax()]['list_broker_name_2']} | Buyer's agents: {merged_df.loc[merged_df['PRICE'].idxmax()]['buy_agent_name_1']} with {merged_df.loc[merged_df['PRICE'].idxmax()]['buy_broker_name_1']} and {merged_df.loc[merged_df['PRICE'].idxmax()]['buy_agent_name_2']} with {merged_df.loc[merged_df['PRICE'].idxmax()]['buy_broker_name_2']}")
print(f"{LE}{BR}{merged_df.loc[merged_df['PRICE'].idxmin()]['LOCATION']}, {merged_df.loc[merged_df['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${merged_df.loc[merged_df['PRICE'].idxmin()]['PRICE']:,.0f} | ${merged_df.loc[merged_df['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {merged_df.loc[merged_df['PRICE'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {merged_df.loc[merged_df['PRICE'].idxmin()]['list_agent_name_1']} with {merged_df.loc[merged_df['PRICE'].idxmin()]['list_broker_name_1']} and {merged_df.loc[merged_df['PRICE'].idxmin()]['list_agent_name_2']} with {merged_df.loc[merged_df['PRICE'].idxmin()]['list_broker_name_2']} | Buyer's agents: {merged_df.loc[merged_df['PRICE'].idxmin()]['buy_agent_name_1']} with {merged_df.loc[merged_df['PRICE'].idxmin()]['buy_broker_name_1']} and {merged_df.loc[merged_df['PRICE'].idxmin()]['buy_agent_name_2']} with {merged_df.loc[merged_df['PRICE'].idxmin()]['buy_broker_name_2']}")

print(f"{MAX_PSF}{BR}{merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['LOCATION']}, {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['list_agent_name_1']} with {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['list_broker_name_1']} and {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['list_agent_name_2']} with {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['list_broker_name_2']} | Buyer's agents: {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['buy_agent_name_1']} with {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['buy_broker_name_1']} and {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['buy_agent_name_2']} with {merged_df.loc[merged_df['$/SQUARE FEET'].idxmax()]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['LOCATION']}, {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['list_agent_name_1']} with {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['list_broker_name_1']} and {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['list_agent_name_2']} with {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['list_broker_name_2']} | Buyer's agents: {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['buy_agent_name_1']} with {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['buy_broker_name_1']} and {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['buy_agent_name_2']} with {merged_df.loc[merged_df['$/SQUARE FEET'].idxmin()]['buy_broker_name_2']}")

print(f"{Newest}{BR}{merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['LOCATION']}, {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['list_agent_name_1']} with {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['list_broker_name_1']} and {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['list_agent_name_2']} with {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['list_broker_name_2']} | Buyer's agents: {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['buy_agent_name_1']} with {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['buy_broker_name_1']} and {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['buy_agent_name_2']} with {merged_df.loc[merged_df['YEAR BUILT'].idxmax()]['buy_broker_name_2']}")
print(f"{Oldest}{BR}{merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['LOCATION']}, {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['list_agent_name_1']} with {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['list_broker_name_1']} and {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['list_agent_name_2']} with {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['list_broker_name_2']} | Buyer's agents: {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['buy_agent_name_1']} with {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['buy_broker_name_1']} and {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['buy_agent_name_2']} with {merged_df.loc[merged_df['YEAR BUILT'].idxmin()]['buy_broker_name_2']}")


HED: Miami-Dade County condo sales dollar volume drops in August to $470M 
DEK: Number of closings fell to 712 from 810 in July
FEATURED HED:
SEO HED: Miami-Dade County August Condo Sales Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales volume drops to $470 million in August.
AUTHOR: Adam Farence
RESEARCH: 
Social: Miami-Dade County had $470 million in condo sales in August, lower than $624 million from July, @afarence reports 
ART: 

CHANGE ME

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, monthly condo sales, condos, condo market, Dinner Key, Miami

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:


Miami-Dade County’s NEWS PEG HERE.

August condo sales totaled $470 million, falling from $624 million in July. Brokers closed 712 sales last month, down from 810 sales in July and XXX in XXX, Multiple Listing Service data from Redfin sho

# Print links for notable sales

## Top sale

In [107]:
print(df_filtered.loc[df_filtered['PRICE'].idxmax()]['URL'])

https://www.redfin.com/FL/Miami/2669-S-Bayshore-Dr-33133/unit-LPH-N/home/190088363


## Cheapest sale

In [108]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami/18801-NE-2nd-Ave-33179/unit-1021/home/43012166


## Highest PSF

In [109]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['URL'])

https://www.redfin.com/FL/Miami-Beach/100-S-Pointe-Dr-33139/unit-3205/home/43372179


## Lowest PSF

In [110]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami/1800-NE-114th-St-33181/unit-1811/home/43012474


## Newest

In [111]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['URL'])




https://www.redfin.com/FL/Miami/2874-SW-33rd-Ct-33133/Unit/home/188465814


## Oldest

In [112]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami-Beach/1027-Pennsylvania-Ave-33139/unit-201/home/43271790


## Time on Market Calculator

In [120]:
from datetime import datetime, timedelta

################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 1, 15) ## List (Earlier) date
date2 = datetime(2024, 8, 1) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

199
